<a href="https://colab.research.google.com/github/Joykareko/AWS-Udacity/blob/main/Training_Batch_transform_processing_job_endpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Training Job

In [ ]:
  #Training Job
  import sagemaker
from sagemaker import get_execution_role
from sagemaker import image_uris
from sagemaker.predictor import csv_serializer

session = sagemaker.Session()

role = get_execution_role()

# If you're following along, you'll need to upload these datasets to your own bucket in S3. 

test_location = 's3://sagemaker-us-west-2-565094796913/boston-xgboost-HL/test.csv'
val_location = 's3://sagemaker-us-west-2-565094796913/boston-xgboost-HL/validation.csv'
train_location = 's3://sagemaker-us-west-2-565094796913/boston-xgboost-HL/train.csv'

# We use this prefix to help us determine where the output will go. 

prefix = 's3://sagemaker-us-west-2-565094796913/'

# We need to get the location of the container. 

container = image_uris.retrieve('xgboost', session.boto_region_name, version='latest')

# Now that we know which container to use, we can construct the estimator object.
xgb = sagemaker.estimator.Estimator(container, # The image name of the training container
                                    role,      # The IAM role to use (our current role in this case)
                                    instance_count=1, # The number of instances to use for training
                                    instance_type='ml.m4.xlarge', # The type of instance to use for training
                                    output_path='s3://{}/{}/output'.format(session.default_bucket(), prefix),
                                                                        # Where to save the output (the model artifacts)
                                    sagemaker_session=session) # The current SageMaker session
             
# These hyperparameters are beyond the scope of this course, but you can research the algoirthm here: 
# https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html    
    
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        objective='reg:linear',
                        early_stopping_rounds=10,
                        num_round=200)
                        
s3_input_train = sagemaker.inputs.TrainingInput(s3_data=train_location, content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data=val_location, content_type='csv')

# The fit method launches the training job. 

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})


### Endpoint

In [ ]:
#endpoint
from sagemaker import get_execution_role
from sagemaker.model import Model
from sagemaker import image_uris

role = get_execution_role()

# You'll need to confirm that this region is located in the same place as the S3 uri of your training job.
# (Check the upper right-hand side of the console.)

image_uri = image_uris.retrieve(framework='xgboost',region='us-west-2', version='latest')

# You'll need to replace this model data with the output S3 uri of your training job. 

model_data = "s3://sagemaker-us-west-2-565094796913/boston-xgboost-HL/output/xgboost-2021-08-31-23-02-30-970/output/model.tar.gz"

model = Model(image_uri=image_uri, model_data=model_data, role=role)

predictor = model.deploy(initial_instance_count=1, instance_type="ml.m5.large")


### Batch Transform Job

In [ ]:
from sagemaker import get_execution_role
from sagemaker.model import Model
from sagemaker import image_uris

role = get_execution_role()

# You'll need to confirm that this region is located in the same place as the S3 uri of your training job.
# (Check the upper right-hand side of the console.)

image_uri = image_uris.retrieve(framework='xgboost',region='us-west-2', version='latest')

# You'll need to replace this with the output uri of a training job. 

model_data = "s3://sagemaker-us-west-2-565094796913/boston-xgboost-HL/output/xgboost-2021-08-31-23-02-30-970/output/model.tar.gz"

# You'll need to replace this with the desired output of your batch transform job. 

batch_transform_output_path = "s3://sagemaker-us-west-2-565094796913/boston-xgboost-HL/test_batch_output-2"

model = Model(image_uri=image_uri, model_data=model_data, role=role)

transformer = model.transformer(
    instance_count=1,
    instance_type='ml.m4.xlarge',
    output_path=batch_transform_output_path
)

# You'll need to replace the output data with your S3 uri of your dataset in S3. 

transformer.transform(
    data="s3://sagemaker-us-west-2-565094796913/boston-xgboost-HL/test.csv",
    data_type='S3Prefix',
    content_type='text/csv',
    split_type='Line'
)


### Processing Job

In [ ]:
#Processing job
%%writefile xgboost_process_script.py

# Execute this cell first to write this script to your local directory. 

import pandas

# This method filters out the column at index 1, which is the crime data. 

def filter_crime_data(input_data_path):
    with open(input_data_path, 'r') as f:
        df = pandas.read_csv(f)
    df.drop(df.columns[[1]], axis=1)
    return df

# The main method takes in data at '/opt/ml/processing/input/data/train.csv' 
# and outputs it as a csv to '/opt/ml/processing/output/data_processed'

if __name__ == "__main__":
    filtered_data = filter_crime_data('/opt/ml/processing/input/data/train.csv')
    filtered_data.to_csv('/opt/ml/processing/output/data_processed')



In [ ]:
import boto3

from sagemaker import get_execution_role
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

role = get_execution_role()

sklearn_processor = SKLearnProcessor(framework_version='0.20.0',
                                     role=role,
                                     instance_type='ml.m5.large',
                                     instance_count=1)


# You will need to replace the 'source' code with the location of the dataset you want to process. 

sklearn_processor.run(code='xgboost_process_script.py',
                        inputs=[ProcessingInput(
                        source='s3://sagemaker-us-west-2-565094796913/boston-xgboost-HL/train.csv',
                        destination='/opt/ml/processing/input/data/')],
                      outputs=[ProcessingOutput(source='/opt/ml/processing/output')]
                     )


In [1]:
#jupyter notebook magic to download data

%mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

--2022-01-14 07:01:04--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘../data/aclImdb_v1.tar.gz’

../data/aclImdb_v1. 100%[===================>]  80.23M  47.1MB/s    in 1.7s    

2022-01-14 07:01:05 (47.1 MB/s) - ‘../data/aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [2]:
import os
import glob

def read_imdb_data(data_dir='../data/aclImdb'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    # Here we represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [3]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

IMDB reviews: train = 12500 pos / 12500 neg, test = 12500 pos / 12500 neg


In [4]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test

In [5]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

IMDb reviews (combined): train = 25000, test = 25000


In [6]:
train_X[100]

'My title ought to be enough.<br /><br />It baffles me that a culture so rich in literary excellence (Dumas, Flaubert, Balzac, Maupassant) would churn out such tosh as the "nouvelle vague" cinematic movement. Until the 20th century, France had a great tradition of artistic lucidity and clever philosophy. But the minute you hand them a movie camera they start acting like WOOOHOOO LOOK HOW WEIRD I CAN BE! PLOT? THEME? PSHAW! LET\'S FILM AN AMUSEMENT PARK RIDE GOING ROUND & ROUND! At least this is not as bad as Godard (who has an unhealthy fascination with the backs of peoples\' heads. Oh-la-la, quel artiste.). No, Truffaut maintains a degree of visual clarity. But so does the security camera at a quickie-mart. The two are indistinguishable.<br /><br />Haha, just as an aside to all you dweeby film school nerds: I bet the vein is popping out the side of your neck right now. But don\'t leave without reading the last sentence of my review.<br /><br />Anyway, if you like French literature, yo

In [7]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import *
stemmer = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
import re
from bs4 import BeautifulSoup

def review_to_words(review):
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

In [9]:
import pickle

cache_dir = os.path.join("../cache", "sentiment_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Preprocess training and test data to obtain words for each review
        #words_train = list(map(review_to_words, data_train))
        #words_test = list(map(review_to_words, data_test))
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        # Write to cache file for future runs
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test

In [10]:
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

Wrote preprocessed data to cache file: preprocessed_data.pkl


In [11]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import joblib
# joblib is an enhanced version of pickle that is more efficient for storing NumPy arrays

def extract_BoW_features(words_train, words_test, vocabulary_size=5000,
                         cache_dir=cache_dir, cache_file="bow_features.pkl"):
    """Extract Bag-of-Words for a given set of documents, already preprocessed into words."""
    
    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = joblib.load(f)
            print("Read features from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Fit a vectorizer to training documents and use it to transform them
        # NOTE: Training documents have already been preprocessed and tokenized into words;
        #       pass in dummy functions to skip those steps, e.g. preprocessor=lambda x: x
        vectorizer = CountVectorizer(max_features=vocabulary_size,
                preprocessor=lambda x: x, tokenizer=lambda x: x)  # already preprocessed
        features_train = vectorizer.fit_transform(words_train).toarray()

        # Apply the same vectorizer to transform the test documents (ignore unknown words)
        features_test = vectorizer.transform(words_test).toarray()
        
        # NOTE: Remember to convert the features using .toarray() for a compact representation
        
        # Write to cache file for future runs (store vocabulary as well)
        if cache_file is not None:
            vocabulary = vectorizer.vocabulary_
            cache_data = dict(features_train=features_train, features_test=features_test,
                             vocabulary=vocabulary)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                joblib.dump(cache_data, f)
            print("Wrote features to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        features_train, features_test, vocabulary = (cache_data['features_train'],
                cache_data['features_test'], cache_data['vocabulary'])
    
    # Return both the extracted features as well as the vocabulary
    return features_train, features_test, vocabulary

In [12]:
# Extract Bag of Words features for both training and test datasets
train_X, test_X, vocabulary = extract_BoW_features(train_X, test_X)

Wrote features to cache file: bow_features.pkl
